In [1]:
##Function to simplify date requests
##Returns a list of strings which can be input into HTML query for different options calls
import requests

def StockFun(tikr):

    tikr = str(tikr)

    headers = {'User-Agent': 'Chrome/46.0.2490.80', 'Content-Type': 'text/html',}
    r = requests.get('https://finance.yahoo.com/quote/'+tikr+'?p='+tikr, headers, timeout=5)
    
    from lxml import html
    tree = html.fromstring(r.content)

    OpEn = tree.xpath('//td[@data-test="OPEN-value"]//text()')[0]#open
    Beta = tree.xpath('//td[@data-test="BETA-value"]//text()')[0]#beta
    day_range = tree.xpath('//td[@data-test="DAYS_RANGE-value"]/text()')[0] #day's range
    f2wk_range = tree.xpath('//td[@data-test="FIFTY_TWO_WK_RANGE-value"]/text()')[0] #52 weeks range
    
    import pandas as pd
    sumdat = pd.DataFrame({'OPEN':OpEn, 'Beta':Beta, 'day range':day_range, '52week range':f2wk_range}, [0])
    
    return sumdat

In [2]:
##Function to simplify date requests
##Returns a list of strings which can be input into HTML query for different options calls

import requests

def OptionsFun(tikr):

    tikr = str(tikr)

    headers = {'User-Agent': 'Chrome/46.0.2490.80', 'Content-Type': 'text/html',}
    r = requests.get('https://finance.yahoo.com/quote/'+tikr+'/options?p='+tikr, headers, timeout=5)
    q = r.status_code

    from bs4 import BeautifulSoup

    soup = BeautifulSoup(r.content, 'lxml')
    s = str(soup)

    #use string serch queries to find desired values
    exp = 'expirationDates":['
    a = s.find(exp)+len(exp)
    #print(a)
    b = s.find(']',a)
    li = s[a:b]
    #print(li)
    end = ','
    #initialize placeholder variables
    s = 0
    e = 0
    pl = []

    while s < len(li) and e != -1:
        beg = s
        e = li.find(end, beg)
        pl = pl + [li[beg:e]]
        s = e+1
    pl[-1]=pl[-1]+li[-1]
    return pl




In [ ]:
## Test function output
#import pandas
tick = 'T'
a = StockFun(tick)
q = a.loc[:,'day range'][0]
print(q)
z = q.find('-')
print(z)
start = q[:z-1]
end = q[z+1:]
start = float(start)
end = float(end)
print(abs(start-end))



In [ ]:
##Function to pull options data from yahoo finance for all listed execution dates using OptionsFun
#Returns csv files labeled by stock symbol and execution data which contain options data including: 
#...strike price, implied volatility, volume, and contract name

import requests
import pandas as pd
import numpy as np

tikr = 'T'
a = StockFun(tikr)
k = OptionsFun(tikr)
OpeN = float(a.loc[:,'OPEN'])
rng = a.loc[:,'day range'][0]
z = rng.find('-')

start = float(q[:z-1])
end = float(q[z+1:])
diff = abs(start-end)

for i in range(len(k)):
    
    r = requests.get('https://finance.yahoo.com/quote/'+tikr+'/options?p='+tikr+'&date='+k[i])

    from lxml import html

    tree = html.fromstring(r.content)
    #read date for options execution
    date = tree.xpath('//span[@class="Fz(s) Mend(10px)"]/text()')[0]
    ##Use requests module with html reader to get list of options
    strike = tree.xpath('//a[@href="/quote/'+tikr+'/options?strike=false&straddle=false"]/text()')  #strike price
    IV = tree.xpath('//td[@class="data-col10 Ta(end) Pstart(7px) Pend(6px)"]/text()') #implied volatility
    volume = tree.xpath('//td[@class="data-col8 Ta(end) Pstart(7px)"]/text()') #volume
    lprice = tree.xpath('//td[@class="data-col3 Ta(end) Pstart(7px)"]/text()') #last price
    contract = tree.xpath('//a[@class="Fz(s) Ell C($c-fuji-blue-1-b)"]/text()') #contract name 'P' indicates put 'C' indicates call
    #print(date)
    ctype = []
    rTurn = []
    bePrice = []
    for i in range(len(contract)):
        strike[i] = float(strike[i])
        lprice[i] = float(lprice[i])
        IV[i] = float(IV[i].strip('%'))/100
        if contract[i].find('C',3) >= 0:
            ctype = ctype + ['CALL']
            rTurn = rTurn + [(OpeN-lprice[i]-strike[i])*100]
            bePrice = bePrice + [lprice[i]+strike[i]]
        else:
            ctype = ctype + ['PUT']
            rTurn = rTurn + [(strike[i]-lprice[i]-OpeN)*100]
            bePrice = bePrice + [strike[i]-lprice[i]]
            
    diff = np.array(strike)-OpeN
  
    file = pd.DataFrame({'b-even_Price':bePrice, 'strike':strike, 'IV':IV, 'volume':volume, 'diff':diff, 'last price':lprice, 'contract':ctype, 'return':rTurn}, index = contract)
    
    #print(options)
    file_name = tikr+'_'+date+'.csv'
    file.to_csv(file_name)


In [8]:
##Function to pull options data from yahoo finance for all listed execution dates using OptionsFun
#Returns csv files labeled by stock symbol and execution data which contain options data including: 
#...strike price, implied volatility, volume, and contract name

import requests
import pandas as pd
import numpy as np
import time

delay = 0
tikr = 'T'
a = StockFun(tikr)
k = OptionsFun(tikr)
OpeN = float(a.loc[:,'OPEN'])
rng = a.loc[:,'day range'][0]
z = rng.find('-')

start = float(rng[:z-1])
end = float(rng[z+1:])
df = abs(start-end)

#print(df)

for i in range(len(k)):
    
    r = requests.get('https://finance.yahoo.com/quote/'+tikr+'/options?p='+tikr+'&date='+k[i])

    from lxml import html

    tree = html.fromstring(r.content)
    #read date for options execution
    date = tree.xpath('//span[@class="Fz(s) Mend(10px)"]/text()')[0]
    ##Use requests module with html reader to get list of options
    strike = tree.xpath('//a[@href="/quote/'+tikr+'/options?strike=false&straddle=false"]/text()')  #strike price
    IV = tree.xpath('//td[@class="data-col10 Ta(end) Pstart(7px) Pend(6px)"]/text()') #implied volatility
    volume = tree.xpath('//td[@class="data-col8 Ta(end) Pstart(7px)"]/text()') #volume
    lprice = tree.xpath('//td[@class="data-col3 Ta(end) Pstart(7px)"]/text()') #last price
    contract = tree.xpath('//a[@class="Fz(s) Ell C($c-fuji-blue-1-b)"]/text()') #contract name 'P' indicates put 'C' indicates call
    #print(date)
    ctype = []
    rTurn = []
    bePrice = []
    riskl = []
    riskh = []
    for i in range(len(contract)):
        strike[i] = float(strike[i])
        lprice[i] = float(lprice[i])
        IV[i] = float(IV[i].strip('%'))/100
        if contract[i].find('C',3) >= 0:
            ctype = ctype + ['CALL']
            rTurn = rTurn + [(OpeN-lprice[i]-strike[i])*100]
            bePrice = bePrice + [lprice[i]+strike[i]]
            riskl = riskl + [(OpeN-df-lprice[i]-strike[i])*100]
            riskh = riskh + [(OpeN+df-lprice[i]-strike[i])*100]

        else:
            ctype = ctype + ['PUT']
            rTurn = rTurn + [(strike[i]-lprice[i]-OpeN)*100]
            bePrice = bePrice + [strike[i]-lprice[i]]
            riskl = riskl + [(strike[i]-lprice[i]-(OpeN-df))*100]
            riskh = riskh + [(strike[i]-lprice[i]-(OpeN+df))*100]

        
    diff = np.array(strike)-OpeN
  
    file = pd.DataFrame({'l_return':riskl, 'h_return':riskh, 'b-even_Price':bePrice, 'strike':strike, 'IV':IV, 'volume':volume, 'diff':diff, 'last price':lprice, 'contract':ctype, 'return':rTurn}, index = contract)

    #print(options)
    file_name = tikr+'_'+date+'.csv'
    file.to_csv(file_name)
    time.sleep(delay)
